In [10]:
import json
from datasets import Dataset

# Load your ShareGPT-style dataset
with open("D:\\Projects\\ByteMentor\\data\\bytementor.training_data_2.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Convert to chat-style plain text entries
chat_texts = []
for conv in data:
    dialogue = conv.get("conversations", [])
    chat = ""
    for turn in dialogue:
        role = turn["from"]
        if role == "human":
            chat += "<|user|>\n" + turn["value"].strip() + "\n"
        elif role == "gpt":
            chat += "<|assistant|>\n" + turn["value"].strip() + "\n"
    chat_texts.append({"text": chat.strip()})

# Convert to HuggingFace dataset
dataset = Dataset.from_list(chat_texts)
dataset.save_to_disk("D:\\Projects\\ByteMentor\\data\\bytementor_chat_ready")


Saving the dataset (0/1 shards):   0%|          | 0/1 [00:00<?, ? examples/s]


OSError: [Errno 22] Invalid argument: 'D:/Projects/ByteMentor/data/bytementor_chat_ready/data-00000-of-00001.arrow'

In [ ]:
from transformers import AutoTokenizer, TrainingArguments, Gemma3ForCausalLM
from trl import SFTTrainer
from peft import LoraConfig
from datasets import load_dataset
from datasets import Dataset
from datasets import load_from_disk

# Load a sample dataset (for example purposes, we use wikitext-2)
train_dataset = load_from_disk("D:\\Projects\\ByteMentor\\data\\bytementor_chat_ready")


# Define the model name
model_name = "google/gemma-3-1b-pt"

# Load the tokenizer and adjust padding settings
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token  # Use eos token as pad token
tokenizer.padding_side = "right"

# Load the Gemma 3 model
model = Gemma3ForCausalLM.from_pretrained(model_name, device_map="cuda")
model.config.use_cache = False  # Disable caching for training

# Set up LoRA configuration for causal language modeling
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["q_proj", "v_proj"],
    bias="none",
    task_type="CAUSAL_LM"
)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",                  # Save directory
    num_train_epochs=3,                      # Start with 3+ epochs (increase if loss improves)
    per_device_train_batch_size=4,           # Increase if you have GPU memory
    gradient_accumulation_steps=4,           # Allows larger effective batch size
    learning_rate=2e-5,                      # Lower LR for stability with LoRA or full fine-tuning
    warmup_steps=100,                        # Gradual warm-up to avoid spikes
    logging_steps=10,                        # Log every 10 steps
    save_steps=500,                          # Save less frequently
    save_total_limit=2,                      # Keep only last 2 checkpoints
    report_to="tensorboard",                 # Enables TensorBoard logging
    group_by_length=True,                    # Efficient packing
    fp16=True,                               # Enable if you're using a GPU with mixed precision (e.g. A100, 3090)
    lr_scheduler_type="cosine"             # Better learning rate schedule
)

# Create the SFTTrainer with LoRA parameters
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    peft_config=lora_config
)
# Fine-tune the model
trainer.train()


# Save the fine-tuned model
model.save_pretrained("./gemma3_finetuned")
tokenizer.save_pretrained("./gemma3_finetuned")

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss


('./gemma3_finetuned\\tokenizer_config.json',
 './gemma3_finetuned\\special_tokens_map.json',
 './gemma3_finetuned\\tokenizer.json')

In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM

# Load model + tokenizer
model = AutoModelForCausalLM.from_pretrained("./gemma3_finetuned")
tokenizer = AutoTokenizer.from_pretrained("./gemma3_finetuned")

# Match training format
prompt = "<|user|>\nWhat is symmetry?\n<|assistant|>\n"
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# Generate response
output = model.generate(
    **inputs,
    max_new_tokens=100,
    do_sample=True,
    top_p=0.9,
    temperature=0.7,
    eos_token_id=tokenizer.convert_tokens_to_ids("<|user|>")  # stop before next question
)

# Only decode newly generated part (after prompt)
generated_tokens = output[0][inputs["input_ids"].shape[1]:]
response = tokenizer.decode(generated_tokens, skip_special_tokens=True)
print(response.strip())


d:\Projects\ByteMentor\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
You have set `use_cache` to `False`, but cache_implementation is set to hybrid. cache_implementation will have no effect.
Some weights of the model checkpoint at ./gemma3_finetuned were not used when initializing Gemma3ForCausalLM: ['model.layers.0.self_attn.q_proj.base_layer.weight', 'model.layers.0.self_attn.q_proj.lora_A.default.weight', 'model.layers.0.self_attn.q_proj.lora_B.default.weight', 'model.layers.0.self_attn.v_proj.base_layer.weight', 'model.layers.0.self_attn.v_proj.lora_A.default.weight', 'model.layers.0.self_attn.v_proj.lora_B.default.weight', 'model.layers.1.self_attn.q_proj.base_layer.weight', 'model.layers.1.self_attn.q_proj.lora_A.default.weight', 'model.layers.1.self_attn.q_proj.lora_B.default.weight', 'model.layers.

IndexError: index out of range in self